In [9]:
import uproot, ROOT, glob, os, random
import matplotlib.pyplot as plt
import numpy as np
import awkward as ak
from sklearn.metrics import roc_curve, roc_auc_score
from joblib import dump, load
from tqdm import tqdm
import pandas as pd
from xgboost import XGBClassifier
import ctypes
import h5py

In [29]:
def flattened_pt_weighted(data, bins, weight):
    weights = np.zeros(len(data))
    pt_hist, bin_edges = np.histogram(data, bins=bins, weights=weight)
    for i in range(len(pt_hist)):
        if pt_hist[i] == 0:
            weights = np.where((data >= bin_edges[i]) & (data < bin_edges[i+1]), 1, weights)
        else:
            weights = np.where((data >= bin_edges[i]) & (data < bin_edges[i+1]), 1/pt_hist[i], weights)

    return weights

def getXS_signal(dsid):
    xs_file = "/cvmfs/atlas.cern.ch/repo/sw/database/GroupData/dev/PMGTools/PMGxsecDB_mc23.txt"
    try:
        with open(xs_file, "r") as f:
            for line in f:
                columns = line.split()
                if columns[0] == str(dsid):
                    return float(columns[2])*float(columns[3])*float(columns[4])
        print( "Couldn't find cross section for dsid", dsid, "so setting to 1.")
    except IOError:
        print("Cross section file not accessible on cvmfs.", dsid, " XS setting to 1.")
    return 1

def getXS_bkg(dsid):
    xs_file = "/cvmfs/atlas.cern.ch/repo/sw/database/GroupData/dev/PMGTools/PMGxsecDB_mc16.txt"
    try:
        with open(xs_file, "r") as f:
            for line in f:
                columns = line.split()
                if columns[0] == str(dsid):
                    return float(columns[2])*float(columns[3])*float(columns[4])
        print( "Couldn't find cross section for dsid", dsid, "so setting to 1.")
    except IOError:
        print("Cross section file not accessible on cvmfs.", dsid, " XS setting to 1.")
    return 1

getXS_bkg(364701)

1894784040.0

In [31]:
%%time
dataset_keys = ["event_id", "ditau_pt", "IsTruthHadronic",
                "f_core_lead", "f_core_subl", "f_subjet_subl", "f_subjets", "f_isotracks",
                "R_max_lead", "R_max_subl", "R_isotrack", "R_tracks_subl",
                "m_core_lead", "m_core_subl", "m_tracks_lead", "m_tracks_subl",
                "d0_leadtrack_lead", "d0_leadtrack_subl",
                "n_track", "n_tracks_lead", "n_tracks_subl", "n_subjets",
                "event_weight", "bdt_score", "bdt_score_new", "eta", "average_mu"]

# Cross Section
bkg_xs = [364701, 364702, 364703, 364704, 364705, 364706,
      364707, 364708, 364709, 364710, 364711, 364712]
signal_xs = [802168]

# File Location
path1 = '/eos/user/j/jlai/bkg/'
path2 = '/eos/user/j/jlai/signal/'


bkg_filelist = []
for index in range(12):
    bkg_filelist.append(path1+f"dijet_flattened_jz{index+1}.h5")
    
signal_filelist = [path2+'boosted_h_flattened.h5']

# Selection Cut
def bkg_cut(df_chunk):
    return ((df_chunk['n_subjets'] >=2) &
           (df_chunk['ditau_pt'] >= 2e5) & (df_chunk['ditau_pt'] <= 1e6) &
            ((df_chunk['n_tracks_lead'] == 1) | (df_chunk['n_tracks_lead'] == 3)) &
             ((df_chunk['n_tracks_subl'] == 1) | (df_chunk['n_tracks_subl'] == 3)))

def signal_cut(df_chunk):
    return ((df_chunk['IsTruthHadronic']==1) & (df_chunk['n_subjets'] >=2) &
            (df_chunk['ditau_pt'] >= 2e5) & (df_chunk['ditau_pt'] <= 1e6) &
            ((df_chunk['n_tracks_lead'] == 1) | (df_chunk['n_tracks_lead'] == 3)) &
             ((df_chunk['n_tracks_subl'] == 1) | (df_chunk['n_tracks_subl'] == 3)))

# Define the chunk size
chunk_size = 10000000  # Adjust this size to suit your system's memory

# Define pT bins for pt weight
pt_bins = np.linspace(200000, 1000000, 41)

def h52panda(filelist, xs, cut, getXS):
    combined = pd.DataFrame()
    
    for index in range(len(filelist)):
        file_path = filelist[index]
    
        # Process the file in chunks
        with h5py.File(file_path, 'r') as h5_file:
            # Determine the total length of the datasets
            total_length = h5_file[dataset_keys[0]].shape[0]

            # Read and process each chunk
            for chunk_start in range(0, total_length, chunk_size):
                chunk_end = chunk_start + chunk_size

                # Use slicing to read a chunk from each dataset in the HDF5 file
                data = {key: h5_file[key][chunk_start:chunk_end] for key in dataset_keys}

                # Convert the dictionary to a pandas DataFrame
                df_chunk = pd.DataFrame(data)

                # Apply Cut
                filtered_chunk = df_chunk[cut(df_chunk)]
                print(f'{filelist[index]}: {total_length}, filtered: {len(filtered_chunk)}')
                
                filtered_chunk = filtered_chunk.copy()
                filtered_chunk.loc[:, 'event_weight'] = filtered_chunk['event_weight'] * getXS(xs[index])
                
                combined = pd.concat([combined, filtered_chunk], ignore_index=True)
    
    combined['pT_weight'] = flattened_pt_weighted(combined['ditau_pt'], pt_bins, combined['event_weight'])
    
    return combined

combined_bkg = h52panda(bkg_filelist, bkg_xs, bkg_cut, getXS_bkg)
combined_bkg.describe()

100%|██████████| 12/12 [00:00<00:00, 63550.06it/s]


/eos/user/j/jlai/bkg/dijet_flattened_jz1.h5: 3963, filtered: 4
/eos/user/j/jlai/bkg/dijet_flattened_jz2.h5: 89999, filtered: 63
/eos/user/j/jlai/bkg/dijet_flattened_jz3.h5: 113492703, filtered: 71030
/eos/user/j/jlai/bkg/dijet_flattened_jz3.h5: 113492703, filtered: 71008
/eos/user/j/jlai/bkg/dijet_flattened_jz3.h5: 113492703, filtered: 70774
/eos/user/j/jlai/bkg/dijet_flattened_jz3.h5: 113492703, filtered: 70914
/eos/user/j/jlai/bkg/dijet_flattened_jz3.h5: 113492703, filtered: 71149
/eos/user/j/jlai/bkg/dijet_flattened_jz3.h5: 113492703, filtered: 70978
/eos/user/j/jlai/bkg/dijet_flattened_jz3.h5: 113492703, filtered: 70863
/eos/user/j/jlai/bkg/dijet_flattened_jz3.h5: 113492703, filtered: 71095
/eos/user/j/jlai/bkg/dijet_flattened_jz3.h5: 113492703, filtered: 70464
/eos/user/j/jlai/bkg/dijet_flattened_jz3.h5: 113492703, filtered: 71138
/eos/user/j/jlai/bkg/dijet_flattened_jz3.h5: 113492703, filtered: 70877
/eos/user/j/jlai/bkg/dijet_flattened_jz3.h5: 113492703, filtered: 25055
/eos/use

,event_id,ditau_pt,IsTruthHadronic,f_core_lead,f_core_subl,f_subjet_subl,f_subjets,f_isotracks,R_max_lead,R_max_subl,...,n_track,n_tracks_lead,n_tracks_subl,n_subjets,event_weight,bdt_score,bdt_score_new,eta,average_mu,pT_weight
count,2.180563e+06,2.180563e+06,2.180563e+06,2.180563e+06,2.180563e+06,2.180563e+06,2.180563e+06,2.180563e+06,2.180563e+06,2.180563e+06,...,2.180563e+06,2.180563e+06,2.180563e+06,2.180563e+06,2.180563e+06,2.180563e+06,2.180563e+06,2.180563e+06,2.180563e+06,2.180563e+06
mean,2.272448e+08,4.079028e+05,3.210180e-06,8.432603e-01,4.359384e-01,6.687770e-02,7.946005e-01,1.649070e-02,9.920412e-02,1.288683e-01,...,5.305563e+00,2.583592e+00,1.640910e+00,2.581474e+00,4.174207e-04,4.889618e-01,8.595786e-02,5.701693e-03,4.160802e+01,6.980098e+00
std,1.449072e+08,1.762375e+05,1.791695e-03,9.269824e-02,9.852765e-01,7.006566e-02,1.072586e-01,6.761751e-02,6.249036e-02,5.030245e-02,...,2.846017e+00,8.120473e-01,9.333030e-01,9.263665e-01,1.300730e-01,8.617235e-02,1.872769e-01,1.266880e+00,1.395234e+01,2.610111e+01
min,8.159050e+05,2.000000e+05,0.000000e+00,1.086599e-02,-9.990000e+02,1.801535e-03,3.916248e-02,0.000000e+00,3.297358e-05,9.343500e-05,...,2.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,4.974442e-16,2.524264e-01,1.220086e-03,-2.499992e+00,5.000000e-01,4.301412e-03
25%,1.137105e+08,2.715688e+05,0.000000e+00,8.051019e-01,2.415380e-01,2.740387e-02,7.328346e-01,5.080736e-03,3.873109e-02,9.160173e-02,...,4.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,9.316649e-06,4.211400e-01,4.129253e-03,-1.078299e+00,3.050000e+01,7.321215e-03
50%,2.006333e+08,3.611798e+05,0.000000e+00,8.626674e-01,4.399914e-01,3.982795e-02,8.138258e-01,1.133802e-02,9.638787e-02,1.365043e-01,...,4.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,3.230758e-05,4.881024e-01,1.382714e-02,4.479362e-03,3.950000e+01,1.362585e-01
75%,3.046194e+08,4.882652e+05,0.000000e+00,9.055358e-01,6.172594e-01,7.208306e-02,8.743883e-01,2.122570e-02,1.587392e-01,1.724881e-01,...,6.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,2.611950e-04,5.510147e-01,5.217577e-02,1.093399e+00,5.250000e+01,1.028887e+00
max,4.930031e+08,9.999968e+05,1.000000e+00,9.984388e-01,1.000000e+00,5.133105e-01,1.067907e+00,3.583736e+01,2.000000e-01,2.000000e-01,...,5.700000e+01,3.000000e+00,3.000000e+00,1.600000e+01,9.602898e+01,8.213419e-01,9.969252e-01,2.499999e+00,9.050000e+01,2.340571e+02


In [33]:
combined_signal = h52panda(signal_filelist, signal_xs, signal_cut, getXS_signal)
combined_signal.describe()

/eos/user/j/jlai/signal/boosted_h_flattened.h5: 15997642, filtered: 3027352
/eos/user/j/jlai/signal/boosted_h_flattened.h5: 15997642, filtered: 1814830


,event_id,ditau_pt,IsTruthHadronic,f_core_lead,f_core_subl,f_subjet_subl,f_subjets,f_isotracks,R_max_lead,R_max_subl,...,n_track,n_tracks_lead,n_tracks_subl,n_subjets,event_weight,bdt_score,bdt_score_new,eta,average_mu,pT_weight
count,4.842182e+06,4.842182e+06,4842182.0,4.842182e+06,4.842182e+06,4.842182e+06,4.842182e+06,4.842182e+06,4.842182e+06,4.842182e+06,...,4.842182e+06,4.842182e+06,4.842182e+06,4.842182e+06,4.842182e+06,4.842182e+06,4.842182e+06,4.842182e+06,4.842182e+06,4.842182e+06
mean,1.000091e+07,5.204682e+05,1.0,9.022052e-01,8.038180e-01,2.204471e-01,9.209372e-01,5.997252e-03,1.890939e-02,2.854451e-02,...,3.367198e+00,1.531918e+00,1.512867e+00,2.121997e+00,7.010893e-14,7.424643e-01,9.177014e-01,-2.873483e-04,4.275169e+01,1.562065e+09
std,5.773868e+06,1.863648e+05,0.0,8.666643e-02,6.657099e-01,1.277059e-01,5.379649e-02,1.671944e-02,3.282849e-02,3.288892e-02,...,1.899286e+00,8.836854e-01,8.733277e-01,4.150824e-01,4.005780e-13,4.790224e-02,1.846273e-01,8.279337e-01,9.711172e+00,3.576257e+09
min,1.000000e+00,2.000025e+05,1.0,1.246108e-03,-9.990000e+02,8.354568e-03,1.365968e-01,0.000000e+00,7.248037e-06,1.008136e-05,...,2.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,2.546348e-18,2.848166e-01,2.457022e-03,-2.499943e+00,1.650000e+01,1.883465e+07
25%,5.002254e+06,3.672420e+05,1.0,8.785414e-01,7.534263e-01,1.088267e-01,8.979499e-01,0.000000e+00,6.064409e-03,1.049362e-02,...,2.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,3.374568e-15,7.179740e-01,9.504632e-01,-5.981661e-01,3.550000e+01,4.762986e+07
50%,1.000289e+07,4.835239e+05,1.0,9.244658e-01,8.600609e-01,2.098723e-01,9.330078e-01,3.092671e-03,1.021083e-02,1.815899e-02,...,3.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,1.252256e-14,7.483552e-01,9.879266e-01,4.898470e-04,4.350000e+01,1.991132e+08
75%,1.500105e+07,6.452794e+05,1.0,9.561875e-01,9.205951e-01,3.262887e-01,9.575755e-01,7.996497e-03,1.712838e-02,3.199333e-02,...,4.000000e+00,3.000000e+00,3.000000e+00,2.000000e+00,4.960119e-14,7.762439e-01,9.959625e-01,5.983117e-01,5.050000e+01,1.087874e+09
max,1.999999e+07,9.999998e+05,1.0,9.995749e-01,1.000000e+00,5.376042e-01,1.097981e+00,1.313532e+01,2.000000e-01,2.000000e-01,...,4.700000e+01,3.000000e+00,3.000000e+00,1.400000e+01,2.201542e-10,8.306370e-01,9.975621e-01,2.499959e+00,6.750000e+01,2.181098e+10


In [34]:
# save panda to csv
path = '/eos/user/j/jlai/ditau/'
combined_bkg.to_csv('combined_bkg_inc_3.csv', index=False)
combined_signal.to_csv('combined_signal_inc_3.csv', index=False)

In [35]:
# track cut
df = combined_bkg
df_1p1p = df[(df['n_tracks_lead'] == 1) & (df['n_tracks_subl'] == 1)]
df_3p3p = df[(df['n_tracks_lead'] == 3) & (df['n_tracks_subl'] == 3)]
df_1p3p = df[
    ((df['n_tracks_lead'] == 1) & (df['n_tracks_subl'] == 3)) |
    ((df['n_tracks_lead'] == 3) & (df['n_tracks_subl'] == 1))
]


len(df_1p1p), len(df_3p3p), len(df_1p3p)

(367706, 612476, 1200381)

In [36]:
df = combined_signal
df_1p1p = df[(df['n_tracks_lead'] == 1) & (df['n_tracks_subl'] == 1)]
df_3p3p = df[(df['n_tracks_lead'] == 3) & (df['n_tracks_subl'] == 3)]
df_1p3p = df[
    ((df['n_tracks_lead'] == 1) & (df['n_tracks_subl'] == 3)) |
    ((df['n_tracks_lead'] == 3) & (df['n_tracks_subl'] == 1))
]


len(df_1p1p), len(df_3p3p), len(df_1p3p)

(2642631, 329969, 1869582)